## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

In [14]:
# Create a list of at least 10 column names to use as X data
x_columns = [
    'Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction', 'YearsAtCompany',
    'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'NumCompaniesWorked', 'YearsWithCurrManager'
]

# Create X_df using your selected columns
X_df = attrition_df[x_columns]

# Show the data types for X_df
X_df.dtypes

,0
Age,int64
DistanceFromHome,int64
Education,int64
EnvironmentSatisfaction,int64
YearsAtCompany,int64
JobInvolvement,int64
JobLevel,int64
JobSatisfaction,int64
NumCompaniesWorked,int64
YearsWithCurrManager,int64


In [15]:
# Split the data into training and testing sets
#from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

In [30]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train = X_train.apply(pd.to_numeric, errors='coerce')
X_test = X_test.apply(pd.to_numeric, errors='coerce')

In [31]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [32]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
department_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_department_encoded = department_encoder.transform(y_train[['Department']])
y_test_department_encoded = department_encoder.transform(y_test[['Department']])

In [23]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition_encoded = attrition_encoder.transform(y_train[['Attrition']])
y_test_attrition_encoded = attrition_encoder.transform(y_test[['Attrition']])

## Part 2: Create, Compile, and Train the Model

In [25]:
# Find the number of columns in the X training data.
num_features = X_train_scaled.shape[1]

# Import the Input object from tensorflow.keras.layers
from tensorflow.keras.layers import Input

# Create the input layer
input_layer = Input(shape=(num_features,))

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(32, activation='relu')(shared_layer1)

In [27]:
# Create a branch for Department
# with a hidden layer and an output layer
# Create the hidden layer for the department branch
department_hidden = layers.Dense(16, activation='relu')(shared_layer2)

# Create the output layer for attrition
attrition_hidden = layers.Dense(16, activation='relu')(shared_layer2)
attrition_output = layers.Dense(y_train_attrition_encoded.shape[1], activation='softmax', name='attrition_output')(attrition_hidden)

# Create the output layer for department
department_output = layers.Dense(y_train_department_encoded.shape[1], activation='softmax', name='department_output')(department_hidden)

In [28]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(16, activation='relu')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(y_train_attrition_encoded.shape[1], activation='softmax', name='attrition_output')(attrition_hidden)

In [37]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model, specifying metrics for each output
model.compile(optimizer='adam',
              loss=['categorical_crossentropy', 'categorical_crossentropy'],
              metrics=[['accuracy'], ['accuracy']]) # Provide a list of metrics for each output

# Summarize the model
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 10)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 64)             │            704 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 32)             │          2,080 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 16)             │            528 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 16)             │            528 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             51 │ dense_3[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             34 │ dense_5[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 3,925 (15.33 KB)

 Trainable params: 3,925 (15.33 KB)

 Non-trainable params: 0 (0.00 B)

In [38]:
# Train the model
history = model.fit(X_train_scaled, [y_train_department_encoded, y_train_attrition_encoded],
                    validation_data=(X_test_scaled, [y_test_department_encoded, y_test_attrition_encoded]),
                    epochs=50, batch_size=32, verbose=1)

Epoch 1/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - attrition_output_accuracy: 0.7908 - attrition_output_loss: 0.5851 - department_output_accuracy: 0.5537 - department_output_loss: 1.0080 - loss: 1.5932 - val_attrition_output_accuracy: 0.8673 - val_attrition_output_loss: 0.3934 - val_department_output_accuracy: 0.6667 - val_department_output_loss: 0.8263 - val_loss: 1.2387
Epoch 2/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - attrition_output_accuracy: 0.8296 - attrition_output_loss: 0.4721 - department_output_accuracy: 0.6529 - department_output_loss: 0.8063 - loss: 1.2785 - val_attrition_output_accuracy: 0.8673 - val_attrition_output_loss: 0.3594 - val_department_output_accuracy: 0.6667 - val_department_output_loss: 0.7658 - val_loss: 1.1486
Epoch 3/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - attrition_output_accuracy: 0.8343 - attrition_output_loss: 0.4299 - department_output_accuracy: 0.6605 - department_output_loss: 0.7626 - loss: 1.1924 - val_attrition_output_accuracy: 0.8673 - va

In [39]:
# Evaluate the model with the testing data
eval_results = model.evaluate(X_test_scaled, [y_test_department_encoded, y_test_attrition_encoded])

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - attrition_output_accuracy: 0.8389 - attrition_output_loss: 0.4574 - department_output_accuracy: 0.6115 - department_output_loss: 0.9791 - loss: 1.4435 


In [40]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {eval_results[3]}")
print(f"Attrition Accuracy: {eval_results[4]}")

Department Accuracy: 0.8435373902320862
Attrition Accuracy: 0.6190476417541504


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. A better metric would be precision, recall, or F1-score, which provide more insight into model performance when predicting rare events like attrition.  For the Department prediction, accuracy might be reasonable since departments likely have a more balanced distribution.
2. Department prediction: Since department classification is a multi-class problem (one employee belongs to one of several departments), softmax ensures that the model outputs probabilities summing to 1 across all department categories.  Attrition prediction: Since the attrition column was one-hot encoded (likely for a binary classification problem), softmax still works but could be replaced with sigmoid for a more direct probability output.
3. Use different evaluation metrics: Incorporate precision, recall, and F1-score to better evaluate performance on imbalanced data like attrition.